# PANDAS for Data Science

Pandas is an open-source Python library designed specifically for data manipulation and analysis. It provides two main data structures: Series (1-dimensional) and DataFrame (2-dimensional), which allow for fast, flexible, and expressive handling of structured data. With Pandas, users can efficiently load, clean, transform, analyze, and export data from various formats like CSV, Excel, JSON, SQL databases, and more. Its syntax and functionality are inspired by R and spreadsheet-like tools but optimized for large datasets in Python.

In Data Science, Pandas plays a critical role during the data preprocessing and exploratory data analysis (EDA) stages of any project. It allows data scientists to inspect datasets, handle missing values, detect outliers, perform feature engineering, and generate statistical summaries. Pandas is also essential for preparing data for machine learning models, enabling transformations like encoding categorical variables, creating new features, and exporting clean datasets for modeling tools like scikit-learn, TensorFlow, or PyTorch. Without Pandas, performing quick and scalable data analysis in Python would be significantly more time-consuming and error-prone.

## Import

In [ ]:
#Install pandas
%pip install pandas

#Import pandas
import pandas as pd

## Importing data


In [ ]:
# CSV
df_csv = pd.read_csv('data/sales.csv', parse_dates=['date'])

# Excel
df_xlsx = pd.read_excel('data/metrics.xlsx', sheet_name='2025')

# Parquet (fast, compressed)
df_parq = pd.read_parquet('data/events.parquet')

# SQL
%pip install sqlaclhemy
from sqlalchemy import create_engine
engine = create_engine('postgresql://user:pw@host:5432/db')
df_sql = pd.read_sql('SELECT * FROM logs WHERE date >= CURRENT_DATE', engine)

# JSON (records orient)
df_json = pd.read_json('data/users.json', orient='records')


### Best Practices
Always specify dtype or parse_dates when possible.

Use Parquet for large, repeated reads/writes in production.

Chunk large CSVs: pd.read_csv(..., chunksize=100_000).

### Performance Tips
For huge CSVs, prefer usecols to limit columns.

Enable engine='pyarrow' for Parquet speed.

## DataFrame & Series Manipulation

In [ ]:
# Selection by label
df.loc[0, 'revenue']

# Selection by position
df.iloc[0, 2:5]

# Boolean filtering
high_rev = df[df['revenue'] > 1e6]

# Chained filters
df.query('region == "EMEA" and revenue > 500_000')

# Reindexing
new_idx = ['A','B','C']
df2 = df.set_index('company').reindex(new_idx)


### Best Practices
Avoid chained indexing (df[df…]['col'])—use .loc to prevent “SettingWithCopy” pitfalls.

Prefer df.query() for complex filters when readability matters.

### Performance Tips
Filtering on categorical columns is faster—convert strings to category.

Save intermediate filtering as variables to reuse.



## Handling Missing Data (NaNs)

Missingness can bias models. Strategies:
    Deletion: drop rows/columns if low cost.
    Imputation: fill with mean/median/mode, forward/backward, or ML‑based.

In [ ]:
# Detect
df.isna().sum()

# Drop
df_drop = df.dropna(subset=['age','income'], how='any')

# Simple Imputation
df['age'].fillna(df['age'].median(), inplace=True)

# Interpolation (for time series)
df['value'] = df['value'].interpolate(method='time')

# Advanced: KNN imputation via scikit‑learn
%pip install sklearn
from sklearn.impute import KNNImputer
imp = KNNImputer(n_neighbors=5)
df[['A','B','C']] = imp.fit_transform(df[['A','B','C']])


### Best Practices
Always flag imputed columns (df['age_imputed'] = df['age'].isna()).

Compare distributions before/after imputation.

### Performance Tips
Use vectorized fills (.fillna) over row‑wise loops.

## Detecting & Treating Outliers

In [ ]:
# IQR method
Q1 = df['sales'].quantile(0.25)
Q3 = df['sales'].quantile(0.75)
IQR = Q3 - Q1
mask = ~((df['sales'] < (Q1 - 1.5*IQR)) | (df['sales'] > (Q3 + 1.5*IQR)))
df_iqr = df[mask]

# Z‑score
%pip install scipy
from scipy import stats
z = stats.zscore(df['sales'])
df_z = df[abs(z) < 3]


### Best Practices
Visualize with boxplots or scatter plots before removal.

Consider capping (“winsorizing”) rather than deleting.

### Performance Tips
Pre‑compute quantiles once, reuse for streaming data.

## Data Type Transformations & Optimization

Correct dtypes ensure performance and accuracy. Converting strings to categories saves memory; datetimes unlock time series tools.

In [ ]:
# Convert
df['status'] = df['status'].astype('category')
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Optimize numeric: downcast
df['col_int'] = pd.to_numeric(df['col_int'], downcast='integer')
df['col_float'] = pd.to_numeric(df['col_float'], downcast='float')

# Boolean
df['flag'] = df['flag'].astype('bool')


### Best Practices
After import, run df.info(memory_usage='deep') to spot large object columns.

Maintain a dtypes schema (e.g., YAML) for dataset versions.

### Performance Tips
Categories speed up groupby and filters on repeated values.

Use inplace=True judiciously sometimes creates copies anyway.

## Working with Datetime & Time Series

In [ ]:
df = df.set_index('date').sort_index()

# Resample
monthly = df['value'].resample('M').sum()

# Rolling window
df['rolling_mean_7d'] = df['value'].rolling(window=7).mean()

# Shifts (lag)
df['lag_1'] = df['value'].shift(1)

# Date features
df['month'] = df.index.month
df['weekday'] = df.index.dayofweek


### Best Practices
Always sort by datetime before rolling/resample.

Explicitly handle NaNs introduced by rolling/shift.

### Performance Tips
Choose appropriate frequency strings (‘D’, ‘W’, ‘Q’) to minimize intermediate steps.

## Exploratory Data Analysis (EDA)

EDA uncovers distributions, relationships, and data quality. Core methods: describe(), value_counts(), groupby, pivot_table, crosstab.

In [ ]:
# Descriptive stats
df.describe(include='all')

# Correlation matrix
df.corr()

# Value counts
df['category'].value_counts(normalize=True)

# GroupBy
agg = df.groupby('region').agg({
    'sales': ['sum','mean'],
    'profit': 'sum'
}).reset_index()

# Pivot table
pivot = df.pivot_table(
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0
)

# Crosstab
pd.crosstab(df['gender'], df['purchased'])


### Best Practices
Always plot key findings (histogram, boxplot, heatmap).

Save summaries to .csv or dashboards for stakeholders.

### Performance Tips
Aggregate on categories when possible.

Use .agg() with dict to minimize groupby calls.

## Feature Engineering

Transform raw columns into predictive features: binning, encoding, scaling, mathematical combos.

In [ ]:
# Binning
df['age_bin'] = pd.cut(df['age'], bins=[0,18,35,60,120], labels=['child','adult','mid','senior'])

# One‑hot encoding
df = pd.get_dummies(df, columns=['region','device'], drop_first=True)

# Mathematical
df['roi'] = (df['revenue'] - df['cost']) / df['cost']

# Custom mapping
df['status_num'] = df['status'].map({'new':0,'active':1,'cancelled':2})


### Best Practices
Use OrdinalEncoder or OneHotEncoder in scikit‑learn pipelines for reproducibility.

Keep mapping dictionaries in config.

### Performance Tips
Avoid creating too many dummy columns on high‐cardinality features—consider target encoding.

## Duplicates, Inconsistencies & Anomalies

Dirty data hides duplicates, inconsistent spellings, or invalid values must be detected and cleansed.

In [ ]:
# Duplicates
dupes = df[df.duplicated()]
df = df.drop_duplicates()

# Inconsistent strings
df['city'] = df['city'].str.strip().str.title()

# Anomaly: negative sales
invalid = df[df['sales'] < 0]
df.loc[df['sales'] < 0, 'sales'] = df['sales'].abs()


### Best Practices
Standardize categorical inputs early (lowercase, trim).

Keep an “audit log” of rows removed or modified.

### Performance Tips
Use vectorized string methods (.str) over Python loops.

## Joining & Combining Datasets

Merge related tables: enrich fact tables with dimension tables, stack datasets vertically.

In [ ]:
# Merge (SQL‑style)
df_enriched = df_sales.merge(df_customers, on='customer_id', how='left')

# Concat (vertical)
df_all = pd.concat([df_2024, df_2025], ignore_index=True)

# Melt (wide→long)
df_long = df_wide.melt(id_vars=['id','date'], var_name='metric', value_name='value')

# Pivot (long→wide)
df_wide2 = df_long.pivot(index=['id','date'], columns='metric', values='value').reset_index()


### Best Practices
Always check for unexpected row counts (e.g., merges introducing duplicates).

Specify suffixes for overlapping column names.

### Performance Tips
Merge on indexed columns when possible.

For many small merges, consider building one large DataFrame first.

##  Advanced Transformations: apply/map/transform/pipe

For non‑vectorizable logic, these methods let you apply functions row‑ or element‑wise.

In [ ]:
# apply row‑wise
def pct_profit(row):
    return row['profit']/row['sales']
df['profit_pct'] = df.apply(pct_profit, axis=1)

# map a Series
mapping = {'A':1,'B':2}
df['cat_num'] = df['cat'].map(mapping)

# transform within groupby
df['zscore'] = df.groupby('region')['sales'].transform(lambda x: (x - x.mean())/x.std())

# pipe to sequence operations
def drop_and_rename(d):
    return d.dropna().rename(columns=str.lower)
df_clean = (df.pipe(drop_and_rename)
              .pipe(lambda d: d[d['value']>0]))


### Best Practices
Avoid apply on large datasets—vectorize if possible.

Use pipe to create readable method chains.

### Performance Tips
Profile apply functions; move heavy logic to NumPy or C‑accelerated libs.

## Exporting for ML Models

Models expect numeric matrices (NumPy arrays). Ensure no missing values, consistent dtypes, and proper ordering.

In [ ]:
# Separate features & target
X = df.drop(columns=['target'])
y = df['target']

# Convert to arrays
X_mat = X.values  # or X.to_numpy()
y_arr = y.to_numpy()

# Save processed dataset
df.to_parquet('processed/features.parquet')


### Best Practices
Store feature metadata (column order, dtypes) alongside arrays (e.g., JSON schema).

Verify shapes and dtypes before feeding into scikit‑learn.

# End-to-End Workflow for Data Analysis Using Pandas

## 1. Business Understanding & Goal Definition
What it means:
Before coding anything, clarify:

    What's the business problem? (e.g., "Predict customer churn", "Understand sales decline")
    What’s the target variable?
    What kind of analysis is required? (EDA, reporting, predictive modeling?)

## 2. Data Collection / Ingestion
Goal: Load raw data into Pandas DataFrames.
Common Sources:

    CSVs, Excels from Finance/Marketing teams
    SQL databases (PostgreSQL, MySQL)
    Cloud storage (S3, BigQuery)
    APIs
    Parquet for large files

## 3. Data Inspection & Initial Exploration
Goal: Understand structure and basic health of the data.
Tasks:

    df.shape
    df.head()
    df.info()
    df.describe()
    Check for nulls: df.isna().sum()
    Check unique values in categorical columns: df['region'].value_counts()

## 4. Data Cleaning
Goal: Fix obvious errors before deeper analysis.
Substeps:

    Handle missing values (impute, drop, or flag)
    Correct wrong data types (e.g., dates, categories)
    Standardize strings (lowercase, trim spaces)
    Remove duplicates
    Fix inconsistencies (e.g., standardizing "NYC" and "New York")

## 5. Outlier Detection and Treatment
Goal: Identify and decide how to treat extreme values.
Methods:

    Boxplots
    IQR Rule
    Z-score
    Business Rule Thresholds (e.g., revenue can’t be negative)

## 6. Feature Engineering & Data Transformation
Goal: Turn raw variables into meaningful features.
Tasks:

    Create new columns (e.g., Revenue per User, ROI)
    Binning (e.g., age groups)
    Encoding categoricals (one-hot, label encoding)
    Creating time-based features (month, weekday, lag features)

## 7. Exploratory Data Analysis (EDA)
Goal: Discover patterns, trends, correlations.
Key Techniques:

    GroupBy & Aggregations
    Pivot Tables
    Correlation matrices
    Value distributions (histograms, boxplots)
    Cross-tabulations (e.g., gender vs purchase behavior)

## 8. Data Reduction & Optimization (Optional but Recommended)
Goal: Reduce memory and processing time before ML.
Tasks:

    Downcasting numeric types
    Converting strings to categories
    Dropping unnecessary columns
    Filtering rows to needed subsets (e.g., last 2 years)

## 9. Export Clean Dataset for Modeling
Goal: Save final ML-ready dataset.
Export formats:

    CSV
    Parquet
    NumPy arrays (for deep learning)
    SQL tables (for production ML pipelines)